In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

ROOT_DIR = Path.cwd().resolve()
if ROOT_DIR.name == "notebooks":
    ROOT_DIR = ROOT_DIR.parent
DATA_PATH = ROOT_DIR / "data"

X_TRAIN_PATH = DATA_PATH / "processed" / "X_train_clean.csv"
Y_TRAIN_PATH = DATA_PATH / "raw" / "y_train.csv"

for p in [X_TRAIN_PATH, Y_TRAIN_PATH]:
    assert p.exists(), f"Missing: {p}"

def read_header_columns(csv_path: Path):
    return list(pd.read_csv(csv_path, nrows=0).columns)

def detect_id_column_x(cols):
    for c in ["Unnamed: 0", "ID", "id"]:
        if c in cols:
            return c
    return None

def detect_id_column_y(cols):
    for c in ["ID", "id", "Unnamed: 0"]:
        if c in cols:
            return c
    return None

def detect_year_column(cols):
    if "YEAR" in cols:
        return "YEAR"
    low = {c.lower(): c for c in cols}
    for cand in ["year", "annee", "année"]:
        if cand in low:
            return low[cand]
    return None

x_cols = read_header_columns(X_TRAIN_PATH)
y_cols = read_header_columns(Y_TRAIN_PATH)

ID_COL_X = detect_id_column_x(x_cols)
ID_COL_Y = detect_id_column_y(y_cols)
YEAR_COL = detect_year_column(x_cols)

assert ID_COL_X is not None
assert ID_COL_Y is not None
assert YEAR_COL is not None

In [ ]:
CHUNKSIZE = 100_000
YEARS = [2015, 2018, 2022]

OUT_X_DIR = DATA_PATH / "processed" / "by_year"
OUT_Y_DIR = DATA_PATH / "raw" / "by_year"
OUT_X_DIR.mkdir(parents=True, exist_ok=True)
OUT_Y_DIR.mkdir(parents=True, exist_ok=True)

X_OUT = {y: OUT_X_DIR / f"X_train_clean_{y}.csv" for y in YEARS}
Y_OUT = {y: OUT_Y_DIR / f"y_train_{y}.csv" for y in YEARS}

In [ ]:
for y in YEARS:
    if X_OUT[y].exists():
        X_OUT[y].unlink()
    if Y_OUT[y].exists():
        Y_OUT[y].unlink()

In [ ]:
def ids_match(x_ids, y_ids):
    xa = pd.to_numeric(x_ids, errors="coerce").to_numpy()
    ya = pd.to_numeric(y_ids, errors="coerce").to_numpy()
    if np.isnan(xa).any() or np.isnan(ya).any():
        return np.array_equal(x_ids.astype(str).to_numpy(), y_ids.astype(str).to_numpy())
    return np.array_equal(xa, ya)

x_iter = pd.read_csv(X_TRAIN_PATH, chunksize=CHUNKSIZE)
y_iter = pd.read_csv(Y_TRAIN_PATH, chunksize=CHUNKSIZE)

x_written = {y: False for y in YEARS}
y_written = {y: False for y in YEARS}

CHECK_EVERY = 20

for i, (x_chunk, y_chunk) in enumerate(zip(x_iter, y_iter), start=1):
    if i == 1 or (CHECK_EVERY and i % CHECK_EVERY == 0):
        if not ids_match(x_chunk[ID_COL_X], y_chunk[ID_COL_Y]):
            raise ValueError(f"IDs not aligned at chunk {i}")

    years = x_chunk[YEAR_COL]
    for y in YEARS:
        mask = years.eq(y)
        if not mask.any():
            continue

        x_part = x_chunk.loc[mask]
        y_part = y_chunk.loc[mask]

        x_part.to_csv(X_OUT[y], mode="a", index=False, header=not x_written[y])
        y_part.to_csv(Y_OUT[y], mode="a", index=False, header=not y_written[y])

        x_written[y] = True
        y_written[y] = True

print("done")


In [ ]:
def count_rows(csv_path: Path, chunksize: int):
    n = 0
    for c in pd.read_csv(csv_path, chunksize=chunksize):
        n += len(c)
    return n

orig_x_rows = count_rows(X_TRAIN_PATH, CHUNKSIZE)
orig_y_rows = count_rows(Y_TRAIN_PATH, CHUNKSIZE)

split_x = {y: count_rows(X_OUT[y], CHUNKSIZE) for y in YEARS}
split_y = {y: count_rows(Y_OUT[y], CHUNKSIZE) for y in YEARS}

df_check = pd.DataFrame({
    "x_rows": pd.Series(split_x),
    "y_rows": pd.Series(split_y),
}).astype(int)

display(df_check)
print("orig_x_rows", orig_x_rows, "sum_split_x", int(df_check["x_rows"].sum()))
print("orig_y_rows", orig_y_rows, "sum_split_y", int(df_check["y_rows"].sum()))


In [ ]:
def unique_years_in_file(x_path: Path, year_col: str, chunksize: int):
    s = set()
    for c in pd.read_csv(x_path, usecols=[year_col], chunksize=chunksize):
        s.update(pd.unique(c[year_col]))
        if len(s) > 3:
            break
    return s

u = {y: unique_years_in_file(X_OUT[y], YEAR_COL, CHUNKSIZE) for y in YEARS}
display(pd.DataFrame({"unique_YEAR_values": pd.Series({k: sorted(list(v)) for k, v in u.items()})}))


In [ ]:
def spotcheck_ids(x_path: Path, y_path: Path, id_x: str, id_y: str, ncheck: int = 100_000):
    x = pd.read_csv(x_path, usecols=[id_x], nrows=ncheck)
    y = pd.read_csv(y_path, usecols=[id_y], nrows=ncheck)
    return ids_match(x[id_x], y[id_y])

res = {y: spotcheck_ids(X_OUT[y], Y_OUT[y], ID_COL_X, ID_COL_Y) for y in YEARS}
display(pd.DataFrame({"ids_aligned_first_rows": pd.Series(res)}))
